In [ ]:
# Datos: https://datosabiertos.bogota.gov.co/dataset/8c2a07f9-5948-41ca-8f7a-447753426ab9/resource/4911565b-4518-4931-93de-d117c10dbdce/download/osb_ocupacion_hosp_ucis.csv

In [ ]:
import requests
import pandas as pd
from google.cloud import storage

In [ ]:
URL = 'https://datosabiertos.bogota.gov.co/dataset/8c2a07f9-5948-41ca-8f7a-447753426ab9/resource/4911565b-4518-4931-93de-d117c10dbdce/download/osb_ocupacion_hosp_ucis.csv'
FILEPATH = './data/camas_bogota_raw.csv'

In [ ]:
r = requests.get( URL, stream = True )

with open( FILEPATH, 'wb' ) as f:
    f.write( r.content )

In [ ]:
camas = pd.read_csv( './data/camas_bogota_raw.csv', encoding = 'ISO-8859-1', dtype = { 'Camas Ocupadas': str, 'Camas habilitadas': str, 'Camas disponibles': str } )

In [ ]:
camas[ 'Camas Ocupadas' ] = camas[ 'Camas Ocupadas' ].apply( lambda x: str( x ).replace( '.', '' ) ).astype( int )
camas[ 'Camas habilitadas' ] = camas[ 'Camas habilitadas' ].apply( lambda x: str( x ).replace( '.', '' ) ).astype( int )
camas[ 'Camas disponibles' ] = camas[ 'Camas disponibles' ].apply( lambda x: str( x ).replace( '.', '' ) ).astype( int )

In [ ]:
camas.dtypes

In [ ]:
camas

In [ ]:
camas_grouped = camas.groupby( 'Servicio' ).agg( { 'Camas Ocupadas': 'sum', 'Camas habilitadas': 'sum', 'Camas disponibles': 'sum' } ).reset_index()

In [ ]:
camas_grouped[ '% Ocupación' ] = camas_grouped[ 'Camas Ocupadas' ] / camas_grouped[ 'Camas habilitadas' ]

In [ ]:
camas_grouped[ 'Servicio' ] = camas_grouped[ 'Servicio' ].replace( { 'Hospitalización General': 'General', 'Unidad de Cuidado Intermedio': 'Intermedio', 'Unidad de cuidado intensivo': 'UCI' } )

In [ ]:
camas_grouped.rename( columns = { '% Ocupación': 'Ocupacion' }, inplace = True )

In [ ]:
ucis = pd.read_csv( 'https://storage.googleapis.com/siscovid/uci_bogota.csv' )

In [ ]:
ucis_num = ucis[ 'Camas UCI ocupadas Covid-19' ].values[ -1 ]

In [ ]:
ucis_num

In [ ]:
listofzeros = [ 0 ] * 3
listofzeros[ list( camas_grouped[ 'Servicio' ].values ).index( 'UCI' ) ] = ucis_num
listofzeros

In [ ]:
camas_grouped[ 'Camas Ocupadas COVID' ] = pd.Series( listofzeros )

In [ ]:
camas_grouped[ 'Camas Ocupadas' ] = camas_grouped[ 'Camas Ocupadas' ] - camas_grouped[ 'Camas Ocupadas COVID' ]

In [ ]:
camas_grouped

In [ ]:
#camas_grouped.to_csv( './data/camas_bogota.csv', index = False )

In [ ]:
#DataFrame to CSV
csv_data=camas_grouped.to_csv(index=False)

#Client, bucket and blob to acces gcloud
client= storage.Client()
bucket= client.get_bucket('siscovid')
blob=bucket.blob('camas_bogota.csv')

#Uploads the file to gcloud
blob.upload_from_string(data=csv_data, content_type='text/csv')